In [1]:
# !pip install mecab-python3

# !pip install unidic-lite
# !pip install japanize-matplotlib

In [3]:
import MeCab
import unidic
import sys
import re
import unicodedata
import matplotlib.pyplot as plt
import tqdm
import japanize_matplotlib
import os
import collections
import glob
import pandas as pd

In [4]:
# Check if MeCab works
# t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

t = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')

sentence = 'すもももももももものうち'
print(t.parse(sentence))

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
EOS



### Here is a translation 
- すもも	名詞 (= noun),一般 (=general),*,*,*,*,すもも,スモモ,スモモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- も	助詞 (=particle) ,係助詞 (=binding particle),*,*,*,*,も,モ,モ
- もも	名詞 (= noun),一般 (=general),*,*,*,*,もも,モモ,モモ
- の	助詞 (=particle) ,連体化(=attributive),*,*,*,*,の,ノ,ノ
- うち	名詞(=noun),非自立(=non-indipendent),副詞可能(adverb)=,*,*,*,うち,ウチ,ウチ

In [5]:
#create the list of meaningless words such as "that" "year" "what" etc... 
with open("stopwords.txt", encoding='UTF-8') as f:
    stopwords = f.read().split("\n")
print(stopwords)

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '別', '話', '私', '屋', '店', '家', '場', '等', '見', '際', '観', 

In [7]:

def count_word(text, pos_tags, stopwords):
    
    words_list = []
    tagger = MeCab.Tagger('-r/nul -d"C:/Program Files (x86)/MeCab/dic/ipadic"')
    
    node = tagger.parseToNode(text)

    while node:
        features = node.feature.split(",")
        base = features[6]
        pos_tag = features[0]
        
        # remove "stopwords"
        if pos_tag in pos_tags and base not in stopwords:
            words_list.append((base, pos_tag))
            

        node = node.next
    count = collections.Counter(words_list)

    return count

# chart
# def draw_barh_chart(y, width):
#     fig, ax = plt.subplots(figsize=(8, 4.5))
#     ax.barh(y, width)

# #     fig.savefig("word_count.png")


# if __name__ == "__main__":
#     # read files
#     #change HERE to try other files
#     with open('F:\Wikipedia\wikiextractor-master\wikiextractor\people\people鈴木みのり (カーリング選手)', 'r', encoding='UTF-8') as files:
#         text = files.read()
    
#     # read stopwords
#     with open("stopwords.txt",encoding='UTF-8') as f:
#         stopwords = f.read().split("\n")

#     pos_tags = ["名詞", "動詞", "形容詞"] 
#     #  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]
#     count = count_word(text, pos_tags, stopwords)

# top100_words = count.most_common(100)
# top100_words
    
# top100 = pd.DataFrame(top100_words, columns = ['word_tuple', "count"]) 
# top100
   
    
    
# Draw a chart
#top_words = count.most_common(10)
#words = list(map(lambda x: x[0], top_words))[::-1]
#counts = list(map(lambda x: x[1], top_words))[::-1]
#draw_barh_chart(words, counts)

In [9]:
file_list = glob.glob("F:/Wikipedia/wikiextractor-master/wikiextractor/princess/*") 

In [10]:
file_list[:5]

['F:/Wikipedia/wikiextractor-master/wikiextractor/princess\\メアリー1世 (イングランド女王)',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/princess\\ヴィクトリア (ドイツ皇后)',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/princess\\アリス (ヘッセン大公妃)',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/princess\\ベアトリス (イギリス王女)',
 'F:/Wikipedia/wikiextractor-master/wikiextractor/princess\\ダイアナ (プリンセス・オブ・ウェールズ)']

In [11]:
# read stopwords
with open("stopwords.txt",encoding='UTF-8') as f:
    stopwords = f.read().split("\n")

pos_tags = ["名詞", "動詞", "形容詞"] 
#  ["名詞", "動詞", "形容詞"]=  ["noun", "verb", "adj"]

In [12]:
dfs = []

In [13]:
for f in file_list:
    with open(f, 'r', encoding='UTF-8') as file:
        text = file.read()
    
    f_name = f.split("\\")[-1]
    
    count = count_word(text, pos_tags, stopwords)
    
    #top100_words = count.most_common(100)
    all_words = count.items()
    
    all_words_df = (
        pd
        .DataFrame(all_words, columns = ['word_tuple', "count"])
        .assign(
            word = lambda x: x["word_tuple"].apply(lambda wt: wt[0]),
            pos = lambda x: x["word_tuple"].apply(lambda wt: wt[1]),
            file_name = f_name
        )
        .sort_values("count", ascending=False)
    )
    #CHANGE PATH!!!!!!!
    full_csv_file = "F:/Wikipedia/wikiextractor-master/wikiextractor/princess/new_csvs/" + f_name + "_words.csv"
    #print(f_name)
    all_words_df.to_csv(full_csv_file, index=None)
    dfs.append(all_words_df)


PermissionError: [Errno 13] Permission denied: 'F:/Wikipedia/wikiextractor-master/wikiextractor/princess\\csv'

In [14]:
overall = (
    pd
    .concat(dfs)
    .groupby(["word", "pos"])
    .pipe(
        lambda grp: pd.DataFrame({
            "total": grp["count"].sum(),
            "articles": grp["file_name"].nunique()
        }))
    .reset_index()
    .sort_values("total", ascending=False)
)

In [16]:
overall[overall.pos == "名詞"].head(60)

,word,pos,total,articles
1872,世,名詞,2221,399
6465,結婚,名詞,1017,401
5824,王,名詞,986,322
2430,公,名詞,985,292
3474,夫,名詞,585,258
3537,妃,名詞,576,265
4186,彼女,名詞,459,158
3571,娘,名詞,448,272
5833,王子,名詞,418,148
456,イギリス,名詞,413,125


In [17]:
overall[overall.pos == "動詞"].head(60)

,word,pos,total,articles
5885,生まれる,動詞,444,339
339,もうける,動詞,179,167
2941,受ける,動詞,178,99
24,いう,動詞,157,75
6889,行う,動詞,143,70
2007,亡くなる,動詞,136,109
3121,呼ぶ,動詞,125,89
4682,持つ,動詞,114,81
5051,暮らす,動詞,98,72
1869,与える,動詞,96,62


In [18]:
overall[overall.pos == "形容詞"].head(60)

,word,pos,total,articles
257,ない,形容詞,264,131
5590,深い,形容詞,55,41
4132,強い,形容詞,44,26
4021,幼い,形容詞,42,34
6562,美しい,形容詞,41,30
6765,良い,形容詞,39,30
6785,若い,形容詞,36,30
5680,無い,形容詞,35,25
8140,高い,形容詞,34,23
4365,悪い,形容詞,34,23


In [19]:
overall.to_csv("princess_text_overall.csv", index= False)

In [20]:
df= pd.read_csv("princess_text_overall.csv")
df

,word,pos,total,articles
0,世,名詞,2221,399
1,結婚,名詞,1017,401
2,王,名詞,986,322
3,公,名詞,985,292
4,夫,名詞,585,258
...,...,...,...,...
8193,尊重,名詞,1,1
8194,尊称,名詞,1,1
8195,尊大,名詞,1,1
8196,尊厳,名詞,1,1
